In [ ]:
!pip install kaggle
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 93% 24.0M/25.7M [00:00<00:00, 129MB/s] 
100% 25.7M/25.7M [00:00<00:00, 118MB/s]


In [ ]:
!unzip imdb-dataset-of-50k-movie-reviews.zip -d ./imdb_dataset


Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: ./imdb_dataset/IMDB Dataset.csv  


In [ ]:
#Step 1, 2,3 and 4
import pandas as pd
import re
import string
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the Dataset
df_imdb = pd.read_csv('./imdb_dataset/IMDB Dataset.csv')
#preprocess the imdb data
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(f'[{string.punctuation}0-9\\r\\t\\n]', ' ', text)  # Remove punctuation and numbers
    text = re.sub('\s+', ' ', text)  # Remove multiple spaces
    return text.strip()

df_imdb['cleaned_text'] = df_imdb['review'].astype(str).apply(preprocess_text)
df_imdb['sentiment'] = df_imdb['sentiment'].map({'positive': 1, 'negative': 0})

df_imdb.to_csv('./imdb_dataset/IMDB_Cleaned.csv', index=False)

# Split Data into Train & Validation Sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_imdb['cleaned_text'].tolist(),
    df_imdb['sentiment'].tolist(),
    test_size=0.2,
    random_state=42
)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load Pre-trained Model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

# Training function
def train(model, train_loader, val_loader, optimizer, epochs=2):
    model.train()
    for epoch in range(epochs):
        total_train_loss = 0
        all_preds, all_labels = [], []

        for batch in train_loader:
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

        train_accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

        print(f"Epoch {epoch + 1}: Training Loss = {total_train_loss / len(train_loader):.4f}")
        print(f"Train Accuracy = {train_accuracy:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}, F1-score = {f1:.4f}")
        evaluate_model(model, val_loader)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    total_val_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            total_val_loss += loss.item()
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

    print(f"Validation Loss = {total_val_loss / len(val_loader):.4f}")
    print(f"Val Accuracy = {val_accuracy:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}, F1-score = {f1:.4f}")
    return val_accuracy, precision, recall, f1

# Train the model
train(model, train_loader, val_loader, optimizer, epochs=3)

# Save trained model
model.save_pretrained("./fine_tuned_distilbert_imdb")
tokenizer.save_pretrained("./fine_tuned_distilbert_imdb")

print("Fine-tuning complete! Model saved.")

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Training Loss = 0.2684
Train Accuracy = 0.8873, Precision = 0.8859, Recall = 0.8887, F1-score = 0.8873
Validation Loss = 0.2078
Val Accuracy = 0.9172, Precision = 0.9237, Recall = 0.9109, F1-score = 0.9173
Epoch 2: Training Loss = 0.1362
Train Accuracy = 0.9500, Precision = 0.9488, Recall = 0.9512, F1-score = 0.9500
Validation Loss = 0.2154
Val Accuracy = 0.9173, Precision = 0.9131, Recall = 0.9238, F1-score = 0.9184
Epoch 3: Training Loss = 0.0549
Train Accuracy = 0.9817, Precision = 0.9809, Recall = 0.9823, F1-score = 0.9816
Validation Loss = 0.2313
Val Accuracy = 0.9119, Precision = 0.8846, Recall = 0.9490, F1-score = 0.9157
Fine-tuning complete! Model saved.


In [ ]:
!pip install huggingface_hub transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained("/content/drive/MyDrive/fine_tuned_distilbert_imdb")
tokenizer.save_pretrained("/content/drive/MyDrive/fine_tuned_distilbert_imdb")


Mounted at /content/drive


('/content/drive/MyDrive/fine_tuned_distilbert_imdb/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_distilbert_imdb/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_distilbert_imdb/vocab.txt',
 '/content/drive/MyDrive/fine_tuned_distilbert_imdb/added_tokens.json')

In [ ]:
!huggingface-cli login


In [ ]:
!ls -lh /ishan/fine_tuned_distilbert_imdb


ls: cannot access '/ishan/fine_tuned_distilbert_imdb': No such file or directory


In [ ]:
from google.colab import files
import shutil

# Compress the model folder
shutil.make_archive("fine_tuned_distilbert_imdb", 'zip', "/content/fine_tuned_distilbert_imdb")

# Download the zipped model
files.download("fine_tuned_distilbert_imdb.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#step 5
#pushing model to hugging face
from huggingface_hub import notebook_login, HfApi

notebook_login()  # Log in to Hugging Face
api = HfApi()
repo_name = "Ishan152/fine_tuned_distilbert_imdb"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)
print("Model uploaded to Hugging Face Hub!")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model uploaded to Hugging Face Hub!


In [ ]:
#https://huggingface.co/Ishan152/fine_tuned_distilbert_imdb

In [ ]:
!pip install fastapi uvicorn transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Set your Hugging Face model repository
MODEL_NAME = "Ishan152/fine_tuned_distilbert_imdb"

# Download and save the model locally
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME)

# Save model to a directory
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

print("✅ Model downloaded and saved locally in 'sentiment_model' folder.")


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

✅ Model downloaded and saved locally in 'sentiment_model' folder.


# New Section